## Backtesting
### Estimación de Eventos en Velas de Precios
En este cuaderno estudiamos la ocurrencia del máximo y el mínimo en una vela de precios a partir de datos de menor resolución. Las características determinadas en este estudio nos pueden valer como estimación de la ocurrencia de dichos eventos cuando tenemos que
hacer suposiciones para simular órdenes limitadas 

In [ ]:
import pickle
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from mplfinance.original_flavor import candlestick2_ohlc
import mplfinance as mpf

In [ ]:
with open('../data/stock_data.pkl', 'rb') as handle:
    stock_data = pickle.load(handle)

In [ ]:
ticker = 'REE'

In [ ]:
stock_df = stock_data['REE'].loc['2010':]
stock_df.head()

Hacemos un muestreo sobre la serie original para construir los datos OHLC en velas 
mensuales.  Estas velas mensuales las utilizaremos para caracterizar la ocurrencia 
de máximos y mínimos

In [ ]:
open_m = stock_df.open.resample('1M').first()
close_m = stock_df.close.resample('1M').last()
high_m = stock_df.high.resample('1M').max()
low_m = stock_df.low.resample('1M').min()

In [ ]:
close_m.head()

In [ ]:
stock_month_df = pd.concat([open_m, high_m, low_m, close_m], axis=1)
stock_month_df.head()

____
Visualizamos las velas a diferente resolución

In [ ]:
view_df = stock_df.loc['2018':'2019']
view_m_df = stock_month_df.loc['2018':'2019']

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18,9))
mpf.plot(view_df, ax=ax1, type='candle', style='yahoo')
mpf.plot(view_m_df, ax=ax2, type='candle', style='yahoo')

____
Si vemos la primera vela mensual. ¿**Podemos determinar qué ocurre primero con datos de menor resolución**?

In [ ]:
windata = view_df.loc[:'2018-01']

In [ ]:
fig, ax = plt.subplots(figsize=(8,4))
ax.axhline(windata.high.max(), c='b')
ax.axhline(windata.low.min(), c='m')
mpf.plot(windata, ax=ax, type='candle', style='yahoo')

In [ ]:
windata.high.max(), windata.high.idxmax()

In [ ]:
windata.low.min(), windata.low.idxmin()

Para una ventana, bastaría determinar cual de los índices ocurre primero

___

In [ ]:
def high_before_low(vseries):
    """ Funcion que """
    return vseries.idxmax() < vseries.idxmin()

In [ ]:
stock_df.resample('1M').apply(high_before_low)

In [ ]:
month_high_before_low = stock_df.resample('1M').apply(high_before_low)['close']
month_high_before_low

____
Si no supieramos la realidad y solo tendríamos la vela mensual, podríamos estimar 
la frecuencia de ocurrencia de los eventos tanto en velas verdes, como en velas rojas

In [ ]:
stock_m_green_candle = stock_month_df.open < stock_month_df.close 
stock_m_green_candle.head(12)

In [ ]:
compare_candle = pd.concat([stock_m_green_candle, month_high_before_low], axis=1)
compare_candle.columns = ['GreenCandle', 'HighBeforeLow']

In [ ]:
pd.crosstab(compare_candle.GreenCandle, compare_candle.HighBeforeLow)

_____
Proporción de **máximos antes que mínimos en velas verdes**

In [ ]:
high_before_green = compare_candle.HighBeforeLow[compare_candle.GreenCandle == True]
high_before_green.value_counts()/high_before_green.shape[0]

Proporción de **máximos antes que mínimos en velas rojas** 

In [ ]:
high_before_red = compare_candle.HighBeforeLow[compare_candle.GreenCandle == False]
high_before_red.value_counts()/high_before_red.shape[0]

Intentamos ver con mas detalle cuales serían las características de ambos casos

In [ ]:
month_openclose_return = stock_month_df.close/stock_month_df.open - 1
month_openclose_return

In [ ]:
compare_candle['opencloseReturn'] = month_openclose_return
compare_candle.boxplot(column='opencloseReturn', by=['GreenCandle','HighBeforeLow'])

____

### Ejercicio Propuesto

1. Calcular el porcentaje del "evento máximo-antes-que-mínimo" para cada uno de las series OHLC mensuales
de los valores del IBEX35, diferenciando por meses de rentabilidad positiva (vela verde) y rentabilidad negativa (vela roja)